<a href="https://colab.research.google.com/github/naman3010/Computer-Vision/blob/main/YOLO_Animal_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of YoLo ALgorithm (YoLo 8 version)

Upload videos of ≤20–30s for faster runs.

1. **Upload a video of animals?**

In [1]:
# 1) Install libraries (Colab)
!pip -q install ultralytics==8.3.31
import os, sys
print('Setup complete.')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.3/886.3 kB 7.2 MB/s eta 0:00:00
Setup complete.


In [2]:
# 2) Imports and model load
from ultralytics import YOLO
import pandas as pd
import numpy as np
import cv2
from IPython.display import Image, Video, display, HTML

# Load a small, fast pretrained model (trianed on COCO dataset)
model = YOLO('yolov8n.pt')  # nano model for speed
class_names = model.names
print('Loaded YOLO model with', len(class_names), 'classes')

# These are 80 classes of COCOdataset...

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 28.5MB/s]


Loaded YOLO model with 80 classes


# New Section

**Here is the list of these 80 classes...**

*Which has most of the daily life useful things/objects...*


{0: 'toaster', 1: 'person', 2: 'bike', 3: 'car', 4: 'motor', 5: 'airplane', 6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'light', 11: 'hydrant', 12: 'sign', 13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'dog', 18: 'cat', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 26: 'hat', 27: 'backpack', 28: 'umbrella', 29: 'shoe', 30: 'eye glasses', 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'skateboard', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove', 41: 'sports ball', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle', 45: 'plate', 46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed', 66: 'mirror', 67: 'dining table', 68: 'window', 69: 'desk', 70: 'toilet', 71: 'door', 72: 'tv', 73: 'stroller', 74: 'mouse', 75: 'remote', 76: 'keyboard', 77: 'scooter', 78: 'microwave',                                                
79: 'other vehicle'}

In [3]:
## 4) Upload a short video (MP4/MOV/AVI format).  # Better to Keep it short for (max 30 seconds)

from google.colab import files
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print('Video uploaded:', video_path)

Saving Animal_Input.mp4 to Animal_Input (1).mp4
Video uploaded: Animal_Input (1).mp4


In [4]:
# 5) Run YOLO on the uploaded video and save an annotated output
out_dir = 'runs_yolo_demo'
os.makedirs(out_dir, exist_ok=True)

pred = model.predict(
    source=video_path,
    conf=0.25,
    imgsz=640,
    save=True,
     project=out_dir,
    name='video_results',
    exist_ok=True
)

# Find the saved annotated video
annotated_video = None
for f in os.listdir(os.path.join(out_dir, 'video_results')):
    if f.lower().endswith(('.mp4','.mov','.avi','.mkv')):
        annotated_video = os.path.join(out_dir, 'video_results', f)
        break

print('Annotated video:', annotated_video)
if annotated_video:
    display(HTML(f"<video src='{annotated_video}' controls width='720'></video>"))



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/538) /content/Animal_Input (1).mp4: 448x640 4 horses, 376.1ms
video 1/1 (frame 2/538) /content/Animal_Input (1).mp4: 448x640 5 horses, 172.7ms
video 1/1 (frame 3/538) /content/Animal_Input (1).mp4: 448x640 4 horses, 176.6ms
video 1/1 (frame 4/538) /content/Animal_Input (1).mp4: 448x640 4 horses, 193.9ms
video 1/1 (frame 5/538) /content/Animal_Input (1).mp4: 448x640 3 horses, 1 cow, 173.8ms
video 1/1 (frame 6/538) /content/Animal_Inpu

In [5]:
# 6) Per-frame and overall class counts from the same predictions
# If you re-run prediction, you can use the returned 'pred' object to iterate results.
frame_class_counts = []  # list of dicts per frame
overall_counts = {name: 0 for name in class_names.values()}

for r in pred:  # each r is a Result for a frame
    labels = r.boxes.cls.cpu().numpy().astype(int) if r.boxes is not None else []
    frame_counts = {}
    for lab in labels:
        cname = class_names.get(lab, f'class_{lab}')
        frame_counts[cname] = frame_counts.get(cname, 0) + 1
    # update overall
    for k, v in frame_counts.items():
        overall_counts[k] = overall_counts.get(k, 0) + v
    frame_class_counts.append(frame_counts)

# Convert per-frame counts into a DataFrame
all_classes = sorted(set().union(*[set(d.keys()) for d in frame_class_counts]) if frame_class_counts else [])
rows = []
for i, d in enumerate(frame_class_counts):
    row = {'frame': i}
    for c in all_classes:
        row[c] = d.get(c, 0)
    rows.append(row)

df_frame_counts = pd.DataFrame(rows)
df_overall = pd.DataFrame([
    {'class': k, 'total_detections': v} for k, v in overall_counts.items() if v>0
], columns=['class', 'total_detections']).sort_values('total_detections', ascending=False)

from IPython.display import display
print('Per-frame class counts:')
display(df_frame_counts.head())
print('\nOverall class totals:')
display(df_overall)

# Save CSVs
csv_dir = os.path.join(out_dir, 'video_results')
per_frame_csv = os.path.join(csv_dir, 'per_frame_class_counts.csv')
overall_csv = os.path.join(csv_dir, 'overall_class_totals.csv')
df_frame_counts.to_csv(per_frame_csv, index=False)
df_overall.to_csv(overall_csv, index=False)
print('Saved per-frame counts to:', per_frame_csv)
print('Saved overall totals to:', overall_csv)

Per-frame class counts:


,frame,cow,elephant,horse,sheep
0,0,0,0,4,0
1,1,0,0,5,0
2,2,0,0,4,0
3,3,0,0,4,0
4,4,1,0,3,0



Overall class totals:


,class,total_detections
0,horse,1745
2,cow,516
1,sheep,9
3,elephant,7


Saved per-frame counts to: runs_yolo_demo/video_results/per_frame_class_counts.csv
Saved overall totals to: runs_yolo_demo/video_results/overall_class_totals.csv


Yes, YOLO can work with videos. When we upload a video, YOLO breaks it into frames and detects objects in each frame one by one. In my case, the video had animals and YOLO successfully detected them throughout the video and generated an annotated output video.
However, YOLO does not understand that the same animal appears across multiple frames. So even though the original video had only 4 horses, YOLO sometimes showed 3, 4, or even 5 horses in different frames. This happens because animals move, overlap, or partially go out of view. To get stable and accurate animal counts, object tracking needs to be added.
